In [9]:
# import statements
import numpy as np
import xarray as xr
import glob
import random
import PolarTestingTrainingSplit_CV
import tensorflow as tf
from tensorflow import keras
import keras_tuner

# Get names of models in which we are testing on
path_to_data = '/home/disk/pna2/aodhan/SurfaceTrendLearning/PoChedleyEtAl2022/TASmaps/*_TrendMaps.nc'
ModelNames = [i[70:-16] for i in glob.glob(path_to_data)]

# Do CV train-test-split 
TrainingPredictorData, TrainingTargetData, TestingPredictorData, TestingTargetData, TestingTotalTrend = PolarTestingTrainingSplit_CV.training_testing_split(path_to_data)


In [10]:
# Reshape target and predictor data 
model_idx = 7
TrainingTargetDataShape = np.shape(TrainingTargetData[model_idx])
TestinTargetDataShape = np.shape(TestingTargetData[model_idx])
TrainingTargetDataReshaped = np.reshape(TrainingTargetData[model_idx], (TrainingTargetDataShape[0], TrainingTargetDataShape[1]*TrainingTargetDataShape[2]))
TestingTargetDataReshaped = np.reshape(TestingTargetData[model_idx], (TestinTargetDataShape[0], TestinTargetDataShape[1]*TestinTargetDataShape[2]))
TrainingTargetDataReshaped = np.transpose([TrainingTargetDataReshaped[:,0], TrainingTargetDataReshaped[:,3]])
TestingTargetDataReshaped = np.transpose([TestingTargetDataReshaped[:,0], TestingTargetDataReshaped[:,3]])
TrainingPredictorDataReshaped = np.reshape(TrainingPredictorData[model_idx], (len(TrainingPredictorData[model_idx]),72,144))
TestingPredictorDataReshaped = np.reshape(TestingPredictorData[model_idx], (len(TestingPredictorData[model_idx]),72,144))

print(np.shape(TrainingPredictorDataReshaped), np.shape(TestingPredictorDataReshaped), np.shape(TrainingTargetDataReshaped), np.shape(TestingTargetDataReshaped))

(8268, 72, 144) (50, 72, 144) (8268, 2) (50, 2)


In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=64, max_value=256, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(72,144,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=8, max_value=64, step=8),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu',
    ),
    keras.layers.MaxPooling2D(
        pool_size=hp.Choice('poolsize', values=[2,3,4,5])
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=8, max_value=128, step=16),
    ),
    keras.layers.Dense(2)
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
              loss='mean_absolute_error',
              metrics=['mean_absolute_error'])
  
  return model

In [12]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_mean_absolute_error',
                          max_trials=20, overwrite=True, directory='/home/disk/p/aodhan/SurfaceTrendLearing/PolarTASApplication/output',project_name="CNN_trial1")
tuner_search.search(TrainingPredictorDataReshaped, TrainingTargetDataReshaped, epochs=4, validation_split=0.1)

Trial 25 Complete [00h 10m 01s]
val_mean_absolute_error: 0.03954349458217621

Best val_mean_absolute_error So Far: 0.03522640839219093
Total elapsed time: 05h 22m 52s

Search: Running Trial #26

Value             |Best Value So Far |Hyperparameter
192               |144               |conv_1_filter
3                 |5                 |conv_1_kernel
32                |40                |conv_2_filter
3                 |5                 |conv_2_kernel
5                 |2                 |poolsize
88                |56                |dense_1_units
0.0001            |0.0001            |learning_rate

Epoch 1/4


133/233 [================>.............] - ETA: 1:05 - loss: 0.0535 - mean_absolute_error: 0.0535